In [ ]:
!pip install -q -U google-generativeai

In [ ]:
!pip install gradio

#### Obtenção da chave API e configuração do Google Generative AI

In [ ]:
import pathlib
import textwrap

import google.generativeai as genai

from IPython.display import display
from IPython.display import Markdown

import gradio as gr


def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [ ]:
from google.colab import userdata

In [ ]:
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

genai.configure(api_key=GOOGLE_API_KEY)

In [ ]:
# Montgem do Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Caminho do arquivo específico no Google Drive
file_path = '/content/drive/MyDrive/base_de_dados_cloud_saude.xlsx'


import pandas as pd

# Leitura de um arquivo Excel usando pandas
df = pd.read_excel(file_path)

In [ ]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from google.colab import drive

# Função para gerar embeddings
def gerar_embeddings(texto):
    result = genai.embed_content(
        model="models/embedding-001",
        content=texto,
        task_type="retrieval_document",
        title="Embedding de uma sintoma")
    return result['embedding']

df['Embeddings'] = df['Descrição Detalhada da Ocorrência'].apply(lambda x: gerar_embeddings(x))

In [ ]:
df.head()

,Local da Ocorrência,Descrição Detalhada da Ocorrência,Orientação,Enviar Ambulância?,Nível de Prioridade,Tipo de Ambulância,Código de Deslocamento,Embeddings
0,"Rua das Flores, 123, apto 102 - Centro","Paciente homem, 55 anos, inconsciente, sem pul...",Realizar RCP (ressuscitação cardiopulmonar) at...,Sim,Vermelho (Nível 1),Unidade de Suporte Avançado (priorizar),"Código 1: Sirene ligada, atendimento com prior...","[0.01772911, -0.010961262, -0.04155377, 0.0071..."
1,"Avenida Brasil, 456, esquina com Rua das Rosas...",Vítima de acidente de trânsito entre carro e m...,"Manter vítimas imóveis, controlar sangramento ...",Sim,Vermelho (Nível 1),Unidade de Suporte Avançado,"Código 1: Sirene ligada, atendimento com prior...","[0.006777581, -0.027838863, -0.0031564625, 0.0..."
2,"Rua do Sol, 789, casa 5 - Jardim Europa","Paciente mulher, 28 anos, com histórico de epi...",Proteger a cabeça do paciente e afastar objeto...,Sim,Laranja (Nível 2),Unidade de Suporte Básico,Código 2: Sem sirene,"[0.04781714, -0.014808956, -0.019456925, 0.043..."
3,"Beco da Felicidade, 10, barraco 2 - Morro do Pelé","Paciente homem, 60 anos, com histórico de asma...",Colocar o paciente em posição confortável e in...,Sim,Laranja (Nível 2),Unidade de Suporte Básico,Código 2: Sem sirene,"[0.041793246, -0.0118380785, -0.04704558, 0.05..."
4,"Praça da Liberdade, s/n, próximo ao monumento ...","Paciente homem, 35 anos, com corte profundo na...",Aplicar pressão direta no corte com um pano li...,Sim,Amarelo (Nível 3),Unidade de Suporte Básico,Código 2: Sem sirene,"[0.03749998, -0.016269883, -0.021911206, 0.018..."


In [ ]:
cosine_similarity(np.array([1,2,3]).reshape(1, -1), np.array([3,5,3]).reshape(1, -1))[0][0] # similaridade

0.8966530605331448

In [ ]:
# Função para diagnosticar com base na similaridade de cosseno
def diagnosticar(descricao_ocorrencia):
    embedding_vitima = gerar_embeddings(descricao_ocorrencia)
    #embedding_vitima = embedding_vitima['embedding']
    embedding_vitima = np.array(embedding_vitima).reshape(1, -1)

    # Calcula a similaridade de cosseno
    #similaridades = df['Embeddings'].apply(lambda x: cosine_similarity(embedding_vitima, np.array(x).reshape(1, -1))[0][0])

    # Lista para armazenar as similaridades
    similaridades = []

    # Calcula a similaridade de cosseno
    for embedding in df['Embeddings']:
        embedding = np.array(embedding).reshape(1, -1)
        similaridade = cosine_similarity(embedding_vitima, embedding)[0][0]
        similaridades.append(similaridade)

    # Converte a lista de similaridades em uma série do pandas
    similaridades = pd.Series(similaridades)

    # Encontra o índice do maior valor de similaridade
    idx_max_similaridade = similaridades.idxmax()

    # Retorna a descrição do diagnóstico e o nome da doença mais similares
    return df.loc[idx_max_similaridade, ["Orientação", "Enviar Ambulância?", "Nível de Prioridade", "Tipo de Ambulância", "Código de Deslocamento"]].to_dict()
    resultado = df.loc[idx_max_similaridade, ["Orientação", "Enviar Ambulância?", "Nível de Prioridade", "Tipo de Ambulância", "Código de Deslocamento"]]
    resultado = resultado.to_frame().reset_index()
    resultado.columns = ['Descrição', 'Detalhe']
    return resultado

In [ ]:
# """sintoma_novo_paciente = "Paciente com dor persistente por mais de 3 meses."
# resultado = diagnosticar(sintoma_novo_paciente)
# print(resultado)"""

In [ ]:
# """sintoma_novo_paciente = "Paciente idosa, 82 anos, com histórico de desidratação crônica, apresentando fraqueza, tontura e boca seca há cerca de 6 horas, recusando-se a se alimentar ou beber líquidos."
# resultado = diagnosticar(sintoma_novo_paciente)
# print(resultado)"""

In [ ]:
def gradio_diagnosticar(sintomas):
    resultado = diagnosticar(sintomas)
    """
    orientacao_medica = resultado["Orientação"]
    enviar_ambulancia = resultado["Enviar Ambulância?"]
    nivel_prioridade = resultado["Nível de Prioridade"]
    tipo_ambulancia = resultado["Tipo de Ambulância"]
    codigo_deslocamento = resultado["Código de Deslocamento"]"""

    return resultado

# gradio_diagnosticar("Paciente homem, 60 anos, com histórico de asma, apresentando falta de ar intensa, tosse e sibilos há cerca de 10 minutos, com uso recente de inalador sem melhora.")

In [ ]:
demo = gr.Interface(
    fn=gradio_diagnosticar,
    inputs="text",
    outputs="dataframe",
    title="Ocorrências do SAMU",
    description="Informe a situação da vítima."
)


# demo.launch()

In [ ]:
from google.colab import userdata

GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY') # GOOGLE_API_KEY: Obtém a chave API do Google armazenada no Colab.

genai.configure(api_key=GOOGLE_API_KEY) # Configura a biblioteca Google Generative AI com a chave API obtida.

In [ ]:
# Create the model
# See https://ai.google.dev/api/python/google/generativeai/GenerativeModel
generation_config = {
  "temperature": 1,
  "top_p": 0.95,
  "top_k": 64,
  "max_output_tokens": 8192,
  "response_mime_type": "text/plain",
}

model = genai.GenerativeModel(
  model_name="gemini-1.5-flash",
  generation_config=generation_config,
  # safety_settings = Adjust safety settings
  # See https://ai.google.dev/gemini-api/docs/safety-settings
)

response = model.generate_content([
  "input: Orientação dada ao solicitante pelo médico regulador, Enviar Ambulância,  Nível de Prioridade, Tipo de Ambulância, Código de Deslocamento",
  "output: A partir do input gere um texto que será um e-mail enviado para as equipes de ambulâncias do SAMU com o título \"Nova Ocorrência\".   Gere um número de protocolo aleatório que irá simular o número de protocolo da ocorrência e adicione ao texto.\n\nTexto sem nenhum tipo de formatação.",
  "input: {\n\"Orientação dada ao solicitante pelo médico regulador\"=\"", "Enviar Ambulância\"=\"Sim\",  \"Nível de Prioridade\"=\"Verde (Nível 4)\", \"Tipo de Ambulância\"=\"Unidade de Suporte Básico\", \"Código de Deslocamento\"=\"Código 2: Sem sirene\"",
  "output: ",
])

print(response.text)
email_body = response.text
print(email_body)


Nova Ocorrência

Número de Protocolo: 987654321

Orientação dada ao solicitante pelo médico regulador: Enviar Ambulância
Nível de Prioridade: Verde (Nível 4)
Tipo de Ambulância: Unidade de Suporte Básico
Código de Deslocamento: Código 2: Sem sirene 

Nova Ocorrência

Número de Protocolo: 987654321

Orientação dada ao solicitante pelo médico regulador: Enviar Ambulância
Nível de Prioridade: Verde (Nível 4)
Tipo de Ambulância: Unidade de Suporte Básico
Código de Deslocamento: Código 2: Sem sirene 



In [ ]:
def gerar_email(sintomas):
  resultado = diagnosticar(sintomas)
  orientacao_medica = resultado["Orientação"]
  enviar_ambulancia = resultado["Enviar Ambulância?"]
  nivel_prioridade = resultado["Nível de Prioridade"]
  tipo_ambulancia = resultado["Tipo de Ambulância"]
  codigo_deslocamento = resultado["Código de Deslocamento"]
  entrada =  f"input: \n\"Orientação dada ao solicitante pelo médico regulador\"=\"{orientacao_medica}\",\n\"Enviar Ambulância\"=\"{enviar_ambulancia}\",  \"Nível de Prioridade\"=\"{nivel_prioridade}\", \"Tipo de Ambulância\"=\"{tipo_ambulancia}\", \"Código de Deslocamento\"=\"{codigo_deslocamento}\""
  entrada=entrada
  response = model.generate_content([
    "input: Orientação dada ao solicitante pelo médico regulador, Enviar Ambulância,  Nível de Prioridade, Tipo de Ambulância, Código de Deslocamento",
    "output: A partir do input gere um texto que será um e-mail enviado para as equipes de ambulâncias do SAMU com o título \"Nova Ocorrência\".   Gere um número de protocolo aleatório que irá simular o número de protocolo da ocorrência e adicione ao texto.\n\nTexto sem nenhum tipo de formatação.",
    entrada,
    "output: ",
  ])

  return response.text


In [ ]:
demo_email = gr.Interface(
    fn=gerar_email,
    inputs="text",
    outputs="text",
    title="Ocorrências do SAMU",
    description="Informe a situação da vítima."
)

#demo_email.launch()
demo_email.launch(debug=True)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://4b557f3131827ca184.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
